In [66]:
import pandas as pd
from sshtunnel import SSHTunnelForwarder
import pymysql
from config import ssh_username, ssh_host, ssh_password, ssh_port, mysql_database, mysql_host, mysql_password, mysql_port, mysql_username
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Establish SSH tunnel
with SSHTunnelForwarder(
    (ssh_host, ssh_port),
    ssh_username=ssh_username,
    ssh_password=ssh_password,
    remote_bind_address=(mysql_host, mysql_port),
) as tunnel:
    print(f'Tunnel local bind port: {tunnel.local_bind_port}')
    print(f'Tunnel is active: {tunnel.is_active}')

    # Connect to MySQL through the tunnel
    conn = pymysql.connect(
        host=mysql_host,
        port=tunnel.local_bind_port,
        user=mysql_username,
        password=mysql_password,
        database=mysql_database,
        connect_timeout=30,  # Increase the connection timeout
    )

    orders = '''
            SELECT order_id, firstname, lastname, email, telephone, payment_city, payment_zone, payment_country, payment_method, 
            shipping_address_1, shipping_city, shipping_country, total, date_added, date_modified, design_file,
            shipping_date FROM oc_order
            '''
    
    order_product = '''
            SELECT order_product_id, order_id, product_id, name, model, quantity, price, total FROM oc_order_product

                     '''
    
    just_product = '''
            SELECT product_id, what, image FROM oc_product

    '''    
    

    orders = pd.read_sql_query(orders, conn)
    order_product = pd.read_sql_query(order_product, conn)
    just_product = pd.read_sql_query(just_product, conn)

    try:
        conn.close()
        print('conn is closed')
    except:
        print('conn still open')
    

df = pd.merge(orders, order_product, on = 'order_id')
df = pd.merge(df, just_product, on = 'product_id')


df = df[(df['firstname'] != '') & (df['lastname'] != '')].reset_index(drop = True)

mlb = [
    'Angels',
    'Astros',
    'Athletics',
    'Blue Jays',
    'Braves',
    'Brewers',
    'Cardinals',
    'Cubs',
    'Diamondbacks',
    'Dodgers',
    'Giants',
    'Indians',
    'Mariners',
    'Marlins',
    'Mets',
    'Nationals',
    'Orioles',
    'Padres',
    'Phillies',
    'Pirates',
    'Rangers',
    'Rays',
    'Red Sox',
    'Reds',
    'Rockies',
    'Royals',
    'Tigers',
    'Twins',
    'White Sox',
    'Yankees',
    'Baseball',
    'MLB'
]


df = df.loc[df['name'].str.contains('|'.join(mlb), case=False)]
# df.to_csv('Baseball_Prospects.csv', index = False)

Tunnel local bind port: 50301
Tunnel is active: True
conn is closed
